### Data preprocessing

In [26]:
import os
# 1. Load
with open("./dataset/podcast.txt", 'r') as f:
    f_podcast = f.read()

f_articles = ''
data_len = int(input("input the number of articles:"))
for i in range(1, data_len):
    path = './dataset/articles/%d.txt' %i
    f = open(path, 'r')
    f_articles += f.read()

input the number of articles:1000


In [27]:
# 2. tokenize lower
import nltk
from nltk import FreqDist
podtokens = nltk.wordpunct_tokenize(f_podcast)
podwords = [w.lower() for w in podtokens]

arttokens = nltk.wordpunct_tokenize(f_articles)
artwords = [w.lower() for w in arttokens]

In [28]:
import re
pattern = re.compile('.*[^a-z].*')

In [29]:
def alphaFreqDist(words):
    alphalist = list()
    for word in words:
        if not pattern.match(word): 
            alphalist.append(word)
    alphafd = FreqDist(alphalist)
    return alphafd

In [30]:
def alphaStopFreqDist(words, stoplist):
    alphalist = list()
    for word in words:
        if not pattern.match(word): 
            if not word in stoplist:
                alphalist.append(word)
    alphafd = FreqDist(alphalist)
    return alphafd

In [31]:
def bigramDist(words, stoplist):
    bilist = list()
    uniDist = alphaStopFreqDist(words, stoplist)
    for i in range(1, len(words)):
        if words[i-1] in uniDist and words[i] in uniDist: 
            biword = words[i-1] + ' ' + words[i] 
            bilist.append(biword)
    biDist = FreqDist(bilist)
    return biDist

In [33]:
p_afdist = alphaFreqDist(podwords)
sorted_afdist = {k:v for k,v in sorted(p_afdist.items(), key=lambda item:item[1], reverse=True)}
for key in list(sorted_afdist.keys())[:20]:
    print(key, sorted_afdist[key])

the 54493
that 50035
to 46133
of 41445
and 36333
a 34858
i 32262
you 29171
it 27498
s 25019
in 23789
is 20611
we 17338
think 14105
so 13315
be 12955
are 12562
have 12175
they 11898
people 11448


In [34]:
a_afdist = alphaFreqDist(artwords)
sorted_afdist = {k:v for k,v in sorted(a_afdist.items(), key=lambda item:item[1], reverse=True)}
for key in list(sorted_afdist.keys())[:20]:
    print(key, sorted_afdist[key])

the 57276
a 34186
of 33124
to 30358
and 28296
in 20474
that 14760
or 14673
is 14392
can 12734
for 11601
with 10948
may 10818
as 9217
people 8480
it 8206
are 7578
this 7369
they 6898
person 6200


In [35]:
stopwords = ['the', 'that', 'd', 'you', 'they', 's', 'to', \
             'be', 'of', 'the', 'in', 'we', 'so', \
             'it', 'was', 'are', 'i', 'am', 'she', \
             'had', 'been', 'is', 'have', 'could', \
             'not', 'her', 'he', 'do', 'and', 'would', \
             'such', 'a', 'his', 'must', 'as', 't', 'or', 're', \
            'this', 'at', 'on', 'with', 'one', 'then', 'if']

In [41]:
asfdist = alphaStopFreqDist(artwords, stopwords)
sorted_asfdist = {k:v for k,v in sorted(asfdist.items(), key=lambda item:item[1], reverse=True)}
for key in list(sorted_asfdist.keys())[:25]:
    print(key, sorted_asfdist[key])

may 10818
people 8480
person 6200
symptoms 5315
also 5118
doctor 4528
cause 3592
treatment 3547
help 3338
blood 3297
cancer 3123
include 3054
pain 2924
body 2686
however 2631
use 2528
risk 2504
disease 2270
effects 2234
drug 2215
skin 2207
health 2170
study 2060
one 2013
medical 1987


In [42]:
asfdist = alphaStopFreqDist(podwords, stopwords)
sorted_asfdist = {k:v for k,v in sorted(asfdist.items(), key=lambda item:item[1], reverse=True)}
for key in list(sorted_asfdist.keys())[:25]:
    print(key, sorted_asfdist[key])

think 14105
people 11448
like 10484
robert 8048
wiblin 8036
would 6277
really 6121
one 5766
things 4933
yeah 4929
kind 4507
lot 4436
going 4249
could 4245
get 4128
say 3618
know 3522
well 3509
actually 3475
something 3355
good 3353
much 3329
work 3307
might 3200
also 2991


In [40]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [43]:
bidist = bigramDist(podwords, stopwords)
sorted_bidict = {k: v for k, v in sorted(bidist.items(), key=lambda item: item[1], reverse=True)}
for key in list(sorted_bidict.keys())[:20]:
    print(key, sorted_bidict[key])

robert wiblin 7970
little bit 716
seems like 671
something like 572
things like 442
machine learning 442
would say 422
effective altruism 414
one thing 386
brian christian 379
david chalmers 348
dave denkenberger 340
hilary greaves 334
many people 322
paul christiano 321
tom kalil 321
katja grace 320
philip tetlock 298
really important 291
glen weyl 291


In [44]:
bidist = bigramDist(artwords, stopwords)
sorted_bidict = {k: v for k, v in sorted(bidist.items(), key=lambda item: item[1], reverse=True)}
for key in list(sorted_bidict.keys())[:20]:
    print(key, sorted_bidict[key])

side effects 1343
may also 1041
blood pressure 776
doctor may 711
may help 611
person may 523
people may 476
immune system 463
may cause 383
united states 380
may experience 355
many people 355
trokendi xr 333
bipolar disorder 322
may include 310
allergic reaction 302
birth control 292
mental health 292
weight loss 289
treatment options 285


In [45]:
from math import *

In [46]:
def mutualinfo(words, stoplist, threshold):
    assoclist = list()
    uniDist = alphaStopFreqDist(words, stoplist)
    for i in range(1, len(words)):
        if (uniDist[words[i-1]] > threshold) and (uniDist[words[i]] > threshold): 
            biword = words[i-1] + ' ' + words[i]
            assoclist.append(biword)
    assocDist = FreqDist(assoclist)
    arDist = FreqDist()
    N = len(words)
    for wordstring in assocDist.keys():
        wordlist = str.split(wordstring, ' ')
        w0 = wordlist[0]
        w1 = wordlist[1]
        ar = (assocDist[wordstring] * N * N) / (1.0 *uniDist[w0] * uniDist[w1]) 
        ar_log2 = log(ar, 2)
        arDist[wordstring] = ar_log2 
    return arDist

In [47]:
p_MIdist = mutualinfo( podwords, stopwords , 2)
# only showing the top 20 scoring bigrams in the dictionary.
sorted_MIdist = {k: v for k, v in sorted(p_MIdist.items(), key=lambda item: item[1], reverse=True)}
for pair in list(sorted_MIdist.keys())[:20]:
    print(pair, sorted_MIdist[pair])

carrick flynn 40.01549128189736
esther duflo 40.01549128189736
cato unbound 40.01549128189736
clay shirky 40.01549128189736
geoffrey irving 39.60045378261852
leo szilard 39.60045378261852
karthik muralidharan 39.60045378261852
yasser arafat 39.60045378261852
lant pritchett 39.60045378261852
deng xiaoping 39.60045378261852
enhanced fermentation 39.60045378261852
louisiana gubernatorial 39.60045378261852
guamanian cuisine 39.60045378261852
el salvador 39.60045378261852
burkina faso 39.60045378261852
warren buffet 39.43052878117621
gitcoin clr 39.43052878117621
destruct timers 39.43052878117621
puerto rico 39.278525687731154
von neumann 39.278525687731154


In [48]:
a_MIdist = mutualinfo( artwords, stopwords , 2)
# only showing the top 20 scoring bigrams in the dictionary.
sorted_MIdist = {k: v for k, v in sorted(a_MIdist.items(), key=lambda item: item[1], reverse=True)}
for pair in list(sorted_MIdist.keys())[:20]:
    print(pair, sorted_MIdist[pair])

lichen planus 39.22619512963619
vas deferens 39.22619512963619
percutaneous nephrolithotomy 39.22619512963619
bok choy 39.22619512963619
owen kramer 39.22619512963619
carnegie mellon 39.22619512963619
locum tenens 39.22619512963619
faraway lands 39.22619512963619
chlorophytum borivilianum 39.22619512963619
epidermal necrolysis 38.81115763035734
hong kong 38.81115763035734
albert einstein 38.81115763035734
vice versa 38.81115763035734
acsm ep 38.81115763035734
dos reis 38.81115763035734
seebri neohaler 38.81115763035734
alana biggers 38.81115763035734
lou gehrig 38.81115763035734
retinitis pigmentosa 38.81115763035734
telogen effluvium 38.81115763035734
